In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import hashlib

In [ ]:
# Importing Classifier Modules
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

In [ ]:
pokemon = pd.read_csv("pokemon.csv") # Dataset pokemon stats
combats = pd.read_csv("combats.csv") # Datas:qXet pokemon battles and who won

pokemon["Type 2"] = pokemon["Type 2"].fillna("NA")
pokemon.head()


In [ ]:
pokemon.columns

# Data preprocessing

## First approach processing

We wanted to make some comparisons according to our beliefs of what is more impactfull in the outcome of a pokemon battle and for that we need a base comparison, we guided this first approach to an approach seen in [here](https://github.com/kartikeya-rana/pokemon_battle/blob/master/Pokemon.ipynb), in this approach the individual opted to create a new data set that consisted of pokemon 1 stats followed by pokemon 2 statistics and finnaly the winner of the battle, 1 in case the first pokemon wins 0 otherwise, for simplicity purposes it is not possible to have a match with no winner.

## Second approach processing

After that we thought about instead of concatenating both pokemons statistics we wanted to make a subtraction of the first pokemon with the second and compare the new results with the first ones, we also eliminated both the generation and the Legendary information, since that we know that pokemons of different generation have the same overall raw power, and we also know that pokemon that are legendary do not have big boosts in their stats, and they do not win against every non legendary pokemon

## Third approach processing

This last approach had in account our knowledge of the domain, we believe that pokemons with higher speed and higher attack are more prone to win

In [ ]:
data_one_hot_encoding = []
i = 0


# for each tuple of combats.csv
for t in combats.itertuples():
    i += 1
    first_pokemon = t[1] # get the first pokemon
    second_pokemon = t[2] # get the second pokemon
    winner = t[3]         # get the winner

    x = pokemon.loc[pokemon["#"]==first_pokemon].values[:, 2:][0] # get info about first pokemon
    y = pokemon.loc[pokemon["#"]==second_pokemon].values[:, 2:][0] # get info about second pokemon
    #diff = (x-y)[:6] # difference between "base stats hp...."
    z = np.concatenate((x,y)) 
    if winner == first_pokemon:
        z = np.append(z, [0])
    else:
        z = np.append(z, [1])
        
        
    data_one_hot_encoding.append(z)
    if i > 5000:
        break
    
data_one_hot_encoding = np.asarray(data_one_hot_encoding)


In [ ]:
data_diff_base_stats = []
i = 0


# for each tuple of combats.csv
for t in combats.itertuples():
    i += 1
    first_pokemon = t[1] # get the first pokemon
    second_pokemon = t[2] # get the second pokemon
    winner = t[3]         # get the winner
    
    x = pokemon.loc[pokemon["#"]==first_pokemon].values[:, 2:][0] # get info about first pokemon
    y = pokemon.loc[pokemon["#"]==second_pokemon].values[:, 2:][0] # get info about second pokemon
    diff = (x[2:9]-y[2:9]) # difference between "base stats hp...."
    
    z = []
    z = np.append(diff,z,0)
    

    if winner == first_pokemon:
        z = np.append(z, [0])
    else:
        z = np.append(z, [1])
        
    data_diff_base_stats.append(z)
    if i > 5000:
        break
    
data_diff_base_stats = np.asarray(data_diff_base_stats)
data_diff_base_stats = data_diff_base_stats[:, :-1].astype(int)

print(data_diff_base_stats[0])
print(pokemon.columns)

In [ ]:
pokemon_with_types = pokemon.copy()
types_list = [x for x in pokemon["Type 2"].unique()] + [x for x in pokemon["Type 1"].unique()]

types_list = list(set(types_list))

print(types_list.remove("NA"))
# make NA -> 0
types_list = ["NA"] + types_list

types_map = { x : types_list.index(x) for x in types_list}

pokemon_with_types["Type 1"] = pokemon_with_types["Type 1"].map(types_map)
pokemon_with_types["Type 2"] = pokemon_with_types["Type 2"].map(types_map)

pokemon.head()

print(types_map)
print(types_list)

In [ ]:
data_with_types = []
i = 0


# for each tuple of combats.csv
for t in combats.itertuples():
    i += 1
    first_pokemon = t[1] # get the first pokemon
    second_pokemon = t[2] # get the second pokemon
    winner = t[3]         # get the winner

    x = pokemon_with_types.loc[pokemon["#"]==first_pokemon]
    x = x.drop(columns=["Name","#"]).values[0]
    y = pokemon_with_types.loc[pokemon["#"]==second_pokemon].drop(columns=["Name","#"]).values[0]
    
    
    z = np.concatenate((x,y)) 
    
    if winner == first_pokemon:
        z = np.append(z, [0])
    else:
        z = np.append(z, [1])
        
    data_with_types.append(z)
    if i > 5000:
        break
    
data_with_types = np.asarray(data_with_types)
data_with_types= pd.DataFrame(data_with_types, columns=[
       'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary' ,
       'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary','Win'] )

data_with_types = data_with_types.astype({
       'Type 1':'int32', 'Type 2':'int32', 'HP':'int32', 'Attack':'int32', 'Defense':'int32', 'Sp. Atk':'int32',
       'Sp. Def':'int32', 'Speed':'int32', 'Generation':'int32', 'Legendary':'int32' ,
       'Type 1':'int32', 'Type 2':'int32', 'HP':'int32', 'Attack':'int32', 'Defense':'int32', 'Sp. Atk':'int32',
       'Sp. Def':'int32', 'Speed':'int32', 'Generation':'int32', 'Legendary':'int32','Win':'int32' })

y = data_with_types["Win"]

print(y)




In [ ]:
data_with_types.head()

### Data correlation number of wins


number of wins is correlated with speed and attack being special attack and defense are also important factors as well as being legendary
Contrary to my beliefs generations are quite well balanced,  since there is almos no correlation between the 2
We can also see that the base stats do not have much correlation with generation, therefore pokemons that belong to diferent generations are on an equal foot in terms of base power.

### Data correlation legendary or not
Once again through this correlation heatmap it is possible to say that generation as no important role to classify a pokemon as legendary, one thing that strikes out is Sp. Attack and Attack as well as Sp. Defense  this seem to be somehow correlated to classifing a pokemon as legendary or not

In [ ]:
unique_ids=pokemon['#']
wins_by_id = []
for _id in unique_ids:
    wins_by_id.append([_id ,(combats["Winner"] == _id).sum() ])
    
wins_by_id = np.asarray(wins_by_id)

pokemon["Wins"] = wins_by_id[:,1]
pokemon["Legendary"] = pokemon["Legendary"].astype(int)




In [ ]:

plt.figure(figsize=(20,8))
sns.heatmap(pokemon.corr(),square=True, vmin=-1, vmax=1, annot=True)

### Data correlation number of wins


number of wins is correlated with speed and attack being special attack and defense are also important factors as well as being legendary
Contrary to my beliefs generations are quite well balanced,  since there is almos no correlation between the 2
We can also see that the base stats do not have much correlation with generation, therefore pokemons that belong to diferent generations are on an equal foot in terms of base power.

### Data correlation legendary or not
Once again through this correlation heatmap it is possible to say that generation as no important role to classify a pokemon as legendary, one thing that strikes out is Sp. Attack and Attack as well as Sp. Defense  this seem to be somehow correlated to classifing a pokemon as legendary or not

# Graph that shows the correlation between number of wins and speed

Overall the speed of the pokemon is more impactfull in the outcome of battle than most of the other attributes.

In [ ]:
#boxplot of Attack vs. Legendary
plt.figure(figsize=(8, 4))
sns.boxplot(x='Legendary',y='Wins',data=pokemon, palette='rainbow')

#stripplot of speed Wins relation
plt.figure(figsize=(20,8))
speed_wins = pokemon[['Speed','Wins']].groupby(['Speed'], as_index=False).max().set_index('Speed')
speed_wins.sort_values(by='Speed',ascending=True).plot(kind='line')
#stripplot of Attack vs. Legendary, palette='rainbow'
plt.figure(figsize=(20,8))
speed_wins = pokemon[['Attack','Wins']].groupby(['Attack'], as_index=False).max().set_index('Attack')
speed_wins.sort_values(by='Attack',ascending=True).plot(kind='line')


# 1 type pokemon go brr cause strong


In [ ]:
type_1 = pokemon[['Type 1','Wins']].groupby(['Type 1'], as_index=False).max().set_index('Type 1')
type_1.sort_values(by='Wins',ascending=False).plot(kind='bar')

type_2 = pokemon[['Type 2','Wins']].groupby(['Type 2'], as_index=False).max().set_index('Type 2')
type_2.sort_values(by='Wins',ascending=False).plot(kind='bar')

In [ ]:
plt.figure(figsize=(48, 12))
plt.xlabel("Name")
plt.ylabel("Wins")
plt.bar(pokemon['Name'][:20], pokemon['Wins'][:20])

In [ ]:

type_1 = pokemon[['Attack','Wins']].groupby(['Attack'], as_index=False).max().set_index('Attack')
type_1.sort_values(by='Wins',ascending=True).plot(kind='bar', figsize=(23,8)).set_ylabel("Wins")
#Number of wins is directly proportional too attack damage

In [ ]:
type_1 = pokemon[['Speed','Wins']].groupby(['Speed'], as_index=False).max().set_index('Speed')
type_1.sort_values(by='Wins',ascending=True).plot(kind='bar', figsize=(40,15))
#Number of wins is directly proportional too Speed
type_1 = pokemon[['Speed','Wins']].groupby(['Speed'], as_index=False).max().set_index('Speed')
type_1.sort_values(by='Speed',ascending=True).plot(kind='line', figsize=(40,15))
#Number of wins is directly proportional too Speed

# Data PreProccessing

This dataset contains categorical features therefore we need to solve this by either mapping it to some number, like we are doing it here or we can use one hot encoding or feature hashing.
One Hot Encoding might be very overwelming as well as feature hashing, since there are 18 different types, here we must include NA since that some pokemons don't have a specified type 2, and if we use this algorithms we need to include a few more columns, in the case of one hot encoding we must include 18 x 2 columns, 36, 36 more columns for our algorithm to use and to calculate coeficients for them.

So we opted with the easy way out and we created an hashmap containg all the unique types and in the end we map those categorical features into values ranging from 0 to 18.




# Classifier Results Not Having in Account Types
As we know types take an important role in pokemon battles, for example a electric pokemon deals 2 times more damage to an water pokemon than to a fire pokemon and to times less damage to a rock pokemon, this interactions might prove challenging to some classification algorithms.

In [ ]:
pokemon.columns
print(data_diff_base_stats[0])
print(data_diff_base_stats[1])
print(data_diff_base_stats[2])
print(data_diff_base_stats[3])
print(data_diff_base_stats[4])

In [ ]:
data_pd_frame = pd.DataFrame(data_diff_base_stats, columns=["HP Diff", "Attack Diff", "Defens Diff", "Sp. Atk diff", "Sp. Def dif", "Speed diff", "Win" ])
for column in data_pd_frame.columns:
    data_pd_frame[column] = data_pd_frame[column].astype(int)

In [ ]:
data_pd_frame.dtypes

In [ ]:
X_diff_stats = data_pd_frame.drop(columns=["Win"]).values

X_diff_train, X_diff_test, y_train, y_test = train_test_split(X_diff_stats , y, test_size=0.25, random_state=42)

In [ ]:

number_of_rows = data_pd_frame.shape[0]
clf = LogisticRegression(max_iter=10000)
clf.fit(X_diff_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_test, y_test) * 100, 2)
print(str(acc_log_reg) + ' percent')




In [ ]:
pd.DataFrame(zip(data_pd_frame.columns, np.transpose(clf.coef_[0])), columns=['features', 'coef']) 

In [ ]:
X_Most_Imp_Features = data_pd_frame.drop(columns=["HP Diff", "Sp. Atk diff", "Sp. Def dif", "Legendary", "Win"])

X_train, X_test, y_train, y_test = train_test_split(X_Most_Imp_Features, y, test_size=0.25, random_state=42)
randomFlorestclf = RandomForestClassifier(n_estimators=100)
model = randomFlorestclf.fit(X_train,y_train)
pred = model.predict(X_test)
# print('Accuracy of {}:'.format(name), accuracy_score(pred, y_test))
print('Accuracy :', accuracy_score(pred, y_test))
print(classification_report(y_test, pred))

In [ ]:

clf = LogisticRegression(C=100,max_iter=10000)
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_test, y_test) * 100, 2)
print(str(acc_log_reg) + ' percent')

In [ ]:

pokemon["Type 1"] = pokemon["Type 1"].map(map_Type_2)
pokemon["Type 2"] = pokemon["Type 2"].map(map_Type_2)

pokemon.head()


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:

clf = RandomForestClassifier(n_estimators=100)
model = clf.fit(X_train[:30500],y_train[:30500])
pred = model.predict(X_test)
# print('Accuracy of {}:'.format(name), accuracy_score(pred, y_test))
print('Accuracy :', accuracy_score(pred, y_test))
print(classification_report(y_test, pred))


pred = model.predict(X_train[30500:])
# print('Accuracy of {}:'.format(name), accuracy_score(pred, y_test))
print('Accuracy :', accuracy_score(pred, y_train[30500:]))
print(classification_report(y_train[30500:], pred))

# Same Dataset but know the winner pokemon stats stay in the first few colums

As we can see next by ordering columns in such way that the stats of the winner pokemon appear first will result in a way worse classifier, therefore is better to not use any process of ordering. Types are included

In [ ]:
data = []
i = 0


# for each tuple of combats.csv
for t in combats.itertuples():
    i += 1
    first_pokemon = t[1] # get the first pokemon
    second_pokemon = t[2] # get the second pokemon
    winner = t[3]         # get the winner

    x = pokemon.loc[pokemon["#"]==first_pokemon].values[:, 2:-1][0] # get info about first pokemon
    y = pokemon.loc[pokemon["#"]==second_pokemon].values[:, 2:-1][0] # get info about second pokemon
    diff = (x-y)[:6] # difference between "base stats hp...."
    
    z = np.concatenate((x,y)) 
    
    if winner == first_pokemon:
        z = np.append(np.concatenate((x,y)), [0])
    else:
        z = np.append(np.concatenate((x,y)), [1])
        
    data.append(z)
    if i > 5000:
        break
    
data = np.asarray(data)
X = data[:, :-1].astype(int)
y = data[:, -1].astype(int)



In [ ]:
print(data[0])

In [ ]:

print(pokemon.columns)

X = data[:, :-1].astype(int)
y = data[:, -1].astype(int)

print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

clf = LogisticRegression(C=0.1,max_iter=10000)
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_test, y_test) * 100, 2)
print(str(acc_log_reg) + ' percent')

pd.DataFrame(zip(data_pd_frame.columns, np.transpose(clf.coef_[0])), columns=['features', 'coef']) 

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
model = clf.fit(X_train, y_train)
pred = model.predict(X_test)
# print('Accuracy of {}:'.format(name), accuracy_score(pred, y_test))
print('Accuracy :', accuracy_score(pred, y_test))
print(classification_report(y_test, pred))




# Same Dataset but know we keep the original place for each pokemon stat


In [ ]:
print(data[0])

In [ ]:


print(pokemon.columns)

X = data[:, :-1].astype(int)
y = data[:, -1].astype(int)

print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

clf = LogisticRegression(C=0.1,max_iter=10000)
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_test, y_test) * 100, 2)
print(str(acc_log_reg) + ' percent')

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, X.shape[1]), random_state=1, max_iter=10000)
clf.fit(X, y)

In [ ]:
model = clf.fit(X_train, y_train)
pred = model.predict(X_test)
# print('Accuracy of {}:'.format(name), accuracy_score(pred, y_test))
print('Accuracy :', accuracy_score(pred, y_test))
print(classification_report(y_test, pred))


In [ ]:
conf_matrix = confusion_matrix(y_test,pred)
print(classification_report(y_test,pred))

In [ ]:
sns.heatmap(conf_matrix, square=True, vmin=-y.shape[0], vmax=y.shape[0], annot=True)


In [ ]:

# limit to categorical data using df.select_dtypes()
data_one_hot_encoding = pd.DataFrame( data_one_hot_encoding,columns=['Type 1', 'Type 2', 'HP', 'Attack',
                                                   'Defense', 'Sp. Atk','Sp. Def', 'Speed',
                                                   'Generation', 'Legendary', 'Type 1', 'Type 2',
                                                   'HP', 'Attack',
                                                   'Defense', 'Sp. Atk','Sp. Def', 'Speed',
                                                   'Generation', 'Legendary', 'Winner'] )
save = data_one_hot_encoding.copy()

data_one_hot_encoding = data_one_hot_encoding.astype({'HP' : 'int32', 'Attack': 'int32',
                                                      'Defense': 'int32','Sp. Atk': 'int32',
                                                      'Sp. Def' : 'int32','Speed' : 'int32',
                                                      'Generation': 'int32', 'Legendary': 'int32', 'Winner': 'int32'})
print(data_one_hot_encoding.dtypes)
X_objects = data_one_hot_encoding.select_dtypes(include=[object])
X_objects.head(3)

In [ ]:

# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()

# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X_objects_fitted = X_objects.apply(le.fit_transform)
X_objects_fitted.head()

# limit to categorical data using df.select_dtypes()
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_objects_fitted)

# 3. Transform
onehotlabels = enc.transform(X_objects_fitted).toarray()
onehotlabels.shape
panda_Hot = pd.DataFrame(onehotlabels)
X_data_one_Hot = data_one_hot_encoding.drop(columns=["Winner", "Type 1", "Type 2"])

X_data_one_Hot.join(panda_Hot)

In [ ]:

print(X_train.shape)
print(y_train.shape)

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, X_data_one_Hot.shape[1]), random_state=1, max_iter=10000)

y = data_one_hot_encoding.Winner
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X_data_one_Hot, y, test_size=0.25, random_state=42)

model = clf.fit(X_train, y_train)
pred = model.predict(X_test)
# print('Accuracy of {}:'.format(name), accuracy_score(pred, y_test))
print('Accuracy :', accuracy_score(pred, y_test))
print(classification_report(y_test, pred))

In [ ]:
X_only_high_cor_features = X_data_one_Hot.drop(columns=["HP", "Sp. Atk", "Sp. Def", "Generation", "Legendary4"])